<a href="https://colab.research.google.com/github/zaeyonz/Fish_distribution/blob/main/Fish_distribution_(Logistic_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Data 준비 (pandas)***

*   read_csv() 함수로 CSV파일을 dataframe으로 변환

In [2]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()         # 처음 5개 행 출력

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
#Species 열에서 고유한 값을 추출  --> unique()사용

print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


---
# ***Species 열을 target으로 설정, 나머지 5개 열은 input data로 사용***

*   input data, target data 생성
---

In [8]:
 # dataframe에서 여러 열을 선택하면 새로운 dataframe이 반환됨, 이를 to_numpy() 메서드로 numpy 배열로 변환하여 fish_input에 저장함

fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

print(fish_input[:5])           # 처음 5개 행 출력 (numpy 배열 상태라서 head() 사용 불가)

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [9]:
# target data 생성

fish_target = fish['Species'].to_numpy()

# ***input data와 target data를 train, test로 분류***
*   train_test_split 사용

In [11]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

*   ***scikit-leearn의 StandardScaler 클래스를 사용하여 train set와 test set를 표준화 전처리 해야함!!!***
*   ***train set의 통계 값으로 test set를 변환해야 함!***

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# ***K-최근접 이웃 분류기의 확률 예측***


In [15]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)            # 최근접 이웃 개수인 k를 3으로 지정
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))         #train set의 점수 확인
print(kn.score(test_scaled, test_target))           #test set의 점수 확인

0.8907563025210085
0.85


*   앞서 fish dataframe에서 7종류의 생선이 있었음
*   **target data를 만들 때 fish['Species']를 사용해 만들었기 때문에 train_target과 test_target에도 7종류의 생선이 들어가 있음**

 ***이렇게 target data에 2개 이상의 클래스가 포함된 문제를 다중 분류 (Multi-class classification)라고 부름***
---

*   앞선 예제에선 target 값을 0과 1로 이진 분류를 하였음.
*   **다중분류에서 또한 숫자로 타깃값을 바꾸어 입력할 수 있지만, scikit-learn에선 문자열 그대로 target 값을 사용 가능함**

*   주의!! --> target 값을 그대로 scikit-learn model에 전달하면 *순서가  자동으로 알파벳 순으로 매겨짐*
*   **따라서 pd.unique(fish['Species'])로 출력했던 순서와 다름!!**

In [16]:
# KNeighborsClassifier에서 정렬된 target 값은 classes_ 속성에 저장되어 있음

print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


↑  'Bream'이 첫 번째 클래스,  'Parkki'가 두 번째 클래스가 되는 식임

In [17]:
# predict() 메서드는 친절하게도 타깃값으로 예측을 출력함

print(kn.predict(test_scaled[:5]))      # test set에 있는 처음 5개 샘플의 target 값을 예측

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


***scikit-learn의 분류 모델은 predict_proba() 메서드로 클래스별 확률값을 반환함***

In [18]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])       #test set에 있는 처음 5개 샘플에 대한 확률을 출력
print(np.round(proba, decimals = 4))            #decimal parameter로 소수점 네 번째 자리까지 표기, 5번째 자리에서 반올림

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [20]:
# 위 모델이 계산한 확률이 가장 가까운 이웃의 비율이 맞는지 확인

distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


*   네 번째 샘플의 이웃은 다섯 번째 클래스인 'Roach'가 1개이고 세 번째 클래스인 'Perch'가 2개임
*   ***따라서 'Roach'의 확률은 1/3 = 0.3333, 'Perch'의 확률은 2/3 = 0.6667임***
*   ***위 모델은 3개의 최근접 이웃을 사용하기 때문에 가능한 확률은 0/3, 1/3, 2/3, 3/3 이 전부임 --> 부족함!!***

# ***Logistic Regression (로지스틱 회귀)***
*   이름은 회귀이지만 분류 모델임.  이 알고리즘은 선형회귀와 동일하게 선형 방정식을 학습함

